<a href="https://colab.research.google.com/github/tarboledah/SyS/blob/main/Parcial3/parcial3_dash_punto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit -q
!pip install pyngrok

In [50]:
%%writefile Punto_1.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Función para calcular los parámetros del sistema masa-resorte-amortiguador
def calcular_parametros(chi, wn):
    k = wn**2  # Constante del resorte (N/m)
    c = 2 * chi * np.sqrt(m * k)  # Coeficiente de amortiguamiento
    return m,c, k

# Función para calcular los polos del sistema
def calcular_polos(chi, wn):
    if chi == 1:
        # Polos reales dobles
        p1 = p2 = -chi * wn
    elif chi < 1:
        # Polos complejos conjugados
        real = -chi * wn
        imag = wn * np.sqrt(1 - chi**2)
        p1 = real + 1j * imag
        p2 = real - 1j * imag
    else:
        # Polos reales distintos
        p1 = -chi * wn + wn * np.sqrt(chi**2 - 1)
        p2 = -chi * wn - wn * np.sqrt(chi**2 - 1)
    return p1, p2

# Función para determinar el tipo de amortiguamiento
def tipo_amortiguamiento(chi):
    if chi == 1:
        return "Amortiguamiento crítico"
    elif chi < 1 and chi > 0:
        return "Subamortiguado"
    elif chi > 1:
        return "Sobreamortiguado"
    elif chi == 0:
        return "Oscilatorio"
    else:
        return "Inestable"

# Función para graficar la respuesta del sistema
def graficar_respuesta_temporal(chi, wn):
    num = [wn**2]
    den = [1, 2*chi*wn, wn**2]

    system = signal.TransferFunction(num, den)

    t, y = signal.step(system)

    fig, ax = plt.subplots()
    ax.plot(t, y)
    ax.set_title("Respuesta temporal del sistema masa-resorte-amortiguador")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Desplazamiento (x)")
    st.pyplot(fig)

# Función para graficar el diagrama de Bode
def graficar_bode(chi, wn):
    num = [wn**2]
    den = [1, 2*chi*wn, wn**2]

    system = signal.TransferFunction(num, den)

    w, mag, phase = signal.bode(system)

    # Módulo en dB
    fig, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(8, 6))

    ax_mag.semilogx(w, mag)  # Bode magnitude plot
    ax_mag.set_title("Diagrama de Bode - Magnitud")
    ax_mag.set_ylabel("Magnitud (dB)")

    # Fase en grados
    ax_phase.semilogx(w, phase)  # Bode phase plot
    ax_phase.set_title("Diagrama de Bode - Fase")
    ax_phase.set_ylabel("Fase (grados)")
    ax_phase.set_xlabel("Frecuencia (rad/s)")

    st.pyplot(fig)

# Función para graficar la respuesta al impulso
def graficar_respuesta_impulso(chi, wn):
    num = [wn**2]
    den = [1, 2*chi*wn, wn**2]

    system = signal.TransferFunction(num, den)

    t, y = signal.impulse(system)

    fig, ax = plt.subplots()
    ax.plot(t, y)
    ax.set_title("Respuesta al Impulso del sistema")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    st.pyplot(fig)


def graficar_respuesta_escalon(chi, wn):
    # Sistema en espacio de estados
    num = [wn**2]
    den = [1, 2*chi*wn, wn**2]

    system = signal.TransferFunction(num, den)

    t, y = signal.step(system)

    fig, ax = plt.subplots()
    ax.plot(t, y)
    ax.set_title("Respuesta a un Escalón del Sistema Masa-Resorte-Amortiguador")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Desplazamiento (x)")
    st.pyplot(fig)

# Interfaz de Streamlit
st.title("Sistema Masa-resorte-amortiguador")

# Selección del factor de amortiguamiento y la frecuencia natural
chi = st.slider("Factor de amortiguamiento (chi):", min_value=-0.5, max_value=2.0, value=1.0, step=0.1)
wn = st.slider("Frecuencia natural (ωn):", min_value=0.1, max_value=10.0, value=1.0, step=0.1)
m = st.slider("Masa (m):", min_value=0.1, max_value=10.0, value=1.0, step=0.1)

# Cálculo de los parámetros mecánicos
m,c, k = calcular_parametros(chi, wn)

# Cálculo de los polos del sistema
p1, p2 = calcular_polos(chi, wn)

# Determinar el tipo de amortiguamiento
amortiguamiento = tipo_amortiguamiento(chi)

# Mostrar resultados
st.subheader("Parámetros del sistema masa-resorte-amortiguador:")
st.write(f"Masa (m): {m} kg")
st.write(f"Coeficiente de amortiguamiento (c): {c:.2f} Ns/m")
st.write(f"Constante del resorte (k): {k:.2f} N/m")

st.subheader("Polos del sistema:")
st.write(f"Polos: {p1:.2f}, {p2:.2f}")

st.subheader("Tipo de amortiguamiento:")
st.write(f"El sistema es: {amortiguamiento}")

# Graficar la respuesta temporal
graficar_respuesta_temporal(chi, wn)

# Graficar los polos en el plano complejo
fig, ax = plt.subplots()
ax.plot([p1.real, p2.real], [p1.imag, p2.imag], 'rx', markersize=12)
ax.axhline(0, color='black',linewidth=0.5)
ax.axvline(0, color='black',linewidth=0.5)
ax.set_title("Polos en el plano complejo")
ax.set_xlabel("Parte Real")
ax.set_ylabel("Parte Imaginaria")
st.pyplot(fig)

# Graficar el diagrama de Bode
st.subheader("Diagrama de Bode")
graficar_bode(chi, wn)

# Graficar la respuesta al impulso
st.subheader("Respuesta al Impulso")
graficar_respuesta_impulso(chi, wn)

# Graficar la respuesta al escalon
st.subheader("Respuesta al escalón")
graficar_respuesta_escalon(chi, wn)


Overwriting Punto_1.py


In [53]:
token='2mrlknjvQjRUIARZ7ocJHYr2JZW_6VQHLEYtmkfeqPhPxyoMA'

In [54]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Punto_1.py --server.port 5011 & #colocar nombre del archivo

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://b07b-34-122-250-184.ngrok-free.app
